# Part2. Neural Networks and Deep Learning

# Chapter 9. Up and Running with Tensorflow

텐서플로는 수치계산을 위한 강력한 오픈소스 하드웨어이다. 대규모 머신러닝에 적합하다. <br>
기본원칙은 간단하다. <br>
1) python에서 계산할 그래프를 정의한 후 텐서플로가 그래프를 가져와서 최적화 된 C++ 코드를 사용하여 효율적으로 실행한다. 
<img src='9-1.PNG'>
중요한 점은 그래프를 여러개의 덩어리로 나누어서 여러개의 CPU나 GPU에서 병렬 수행이 가능하다. 
<img src='9-2.PNG'>
텐서플로는 분산 컴퓨팅을 지원하기 때문에 엄청난 양의 학습 데이터를 훈련 시킬 수 있다. <br>
또한, 수십억개의 인스턴스로 구성된 트레이닝 세트의 수백만개의 파라미터가 있는 네트워크를 학습 시킬 수 있다. 

### 텐서플로의 주요 특징
1. 윈도우, 리눅스, 맥 OS 뿐만 아니라 iOS 및 안드로이드를 포함한 모바일 장치에서도 실행이 가능하다. 
2. Scikit-learn과 호환되는 TF.Learn2라는 매우 간단한 파이썬 API를 제공한다. 몇 줄 만으로 다양한 유형의 신경망을 훈련시킬 수 있다.
3. TF-slim이라는 API를 제공하여 신경망을 빌드하고, 학습, 평가하는 작업을 단순화하였다. 
4. Keras 또는 Pretty Tensor와 같이 텐서플로우 위에 여러가지 고수준의 API를 독립적으로 구추하였다. 
5. 주요 파이썬 API는 신경망 아키텍처를 포함하여 모든 종류의 계산을 훨씬 유연하게 제공한다. 
6. 뉴럴 네트워크를 빌드하기 위해 필요한 많은 ML작업의 고효율적인 C++ 구현이 포함된다. 
7. 비용함수(cost function)를 최소화하는 파라미터를 검색할 수 있는 고급 최적화 노드를 제공한다. 
8. TensorBoard라는 훌륭항 시각화 도구가 있어 계산 그래스를 탐색하고 학습 곡선을 보는 등의 작업을 수행할 수 있다. 
9. Google은 텐서플로우 그래프를 실행하기 위해 클라우드 서비스를 시작했다. 

이번장은 설치, 간단한 계산 그래프 생성, 실행, 저장 및 시각화에 이르는 텐서플로의 기본 사항을 살펴본다. 

# Setup

First, let's make sure this notebook works well in both python 2 and 3, import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures:

In [52]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# 1. Creating and running a graph

아래의 코드는 다음 그림을 표현한다.
<img src='9-1.PNG'>

In [53]:
import tensorflow as tf

reset_graph()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [54]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

지금의 상태는 변수 초기화도 안된 상태로 그래프를 생성하기만 한다. <br>
그래프의 평가를 위해서는 텐서플로의 session()을 열고 변수를 초기화해야한다. <br>
세션을 이용한 다음 닫아주면서 마무리한다.

In [55]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [56]:
sess.close()

하지만 매번 session을 열고 닫는 작업이 번거워 with 구문을 제공한다. <br>

with 블록의 세션은 기본 default 세션으로 설정된다. <br>
**x.initializer.run()** 은 tf.get_default_session().run과 같고 <br>
**f.eval()** 은 tf.get_default_session().run(f)와 같다.  <br>
그리고 with를 사용하면 세션은 자동적으로 닫힌다. (장점)


In [57]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [6]:
result

42

단일 변수마다 초기화를 하는 것 대신 global_variables_initalizer() 함수를 사용하여 모든 변수를 초기화 할 노드를 만든다. 

In [58]:
init = tf.global_variables_initializer() # init 노드를 준비

with tf.Session() as sess:
    init.run()  # 생성한 init 노드를 실행 -> 모든 변수를 초기화
    result = f.eval()

In [8]:
result

42

jupyter notebook이나 python에서 interactiveSession을 만드는 것이 더 좋다. <br>
일반 세션과 다른점은 InteractiveSession이 생성되면 기본세션이 자동적으로 생성되어 with 블록이 필요하지 않다. (세션은 수동으로 닫아야 함)

In [9]:
init = tf.global_variables_initializer()

In [10]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


In [11]:
sess.close()

In [12]:
result

42

텐서플로는 크게 두부분으로 나뉜다.
#### 1. Construction Phase : ML 모델과 이를 학습시키기 위한 계산을 나타내는 계산 그래프 빌드 부분
#### 2. Execution Phase : 학습을 반복적으로 평가하는 루프를 실행하여 점진적으로 모델 파라미터를 향상시키는 실행 부분 

# 2. Managing graphs
우리가 생성한 노드는 default 그래프에 자동적으로 추가된다.

In [14]:
reset_graph()

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

대부분의 경우 잘 맞지만 때때로 여러개의 그래프를 관리해야 할 수도 있다. <br>
새로운 그래프를 생성하고 일시적으로 그래프를 with 블록안에 만들어야 한다면 아래처럼 하면 된다. 

In [15]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [16]:
x2.graph is tf.get_default_graph()

False

# 3. Lifecycle of a Node Value
노드 평가시 텐서플로는 의존적인 노드 집합을 자동으로 결정하고 그 노드를 먼저 평가한다. 

In [17]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

10
15


텐서플로는 y가 x에 의존적이라는 것을 감지하고 x는 w에 의존적이라는 것을 감지한다. <br>
즉 w를 먼저 평가하고 x, y를 순차적으로 값을 반환한다. <br>
또 z를 반환하기 위해 또다시 w와 x를 감지하게 되는데 이때 w,x는 두번 계산이 된다. <br>

y와 z를 효율적으로 평가하기 위한 방법은 다음과 같다. <br>
y와 z를 한 그래프에서 동시에 평가하면 된다. 

In [18]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


# 4. Linear Regression

## 4.1 Using the Normal Equation
텐서플로우 operation(ops)은 임의의 수의 입력값을 가져와서 여러개의 출력값을 생성할 수 있다. <br>
덧셈과 곱셈은 두개의 입력값으로 하나의 출력값을 생성한다. 상수와 변수는 입력값은 없다. <br>

입력과 출력은 다차원 배열이며 이를 tensor라 부른다. Numpy의 배열과 마찬가지로 텐서는 type과 shape을 가진다. <br>
Python의 API tensor는 Numpy의 ndarray로 표현된다. 

지금까지는 tensor가 단순히 하나의 스칼라 값을 포함하고 있었지만 어떤 shape의 배열도 계산을 수행할 수 있다. <br>

#### 아래의 코드는 2차원 배열을 조작하여 캘리포니아 주택 데이터 세트의 선형회귀를 수행한다. <br>

데이터 셋을 불러와 모든 학습 인스턴스에 extra bias input feature($x_0=1$)을 추가한다. <br>
텐서플로 상수 노드인 X와 y를 생성하고 각각 데이터와 target을 넣어준다. <br>
theta를 정의하기 위해 텐서플로에서 제공하는 matrix operation을 사용한다.<br>
(이때의 theta는 $\hat\theta={(X^T\cdot X)}^{-1}\cdot X^T\cdot y$ 정규방정식과 일치한다.)


In [19]:
import numpy as np
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [20]:
theta_value

array([[ -3.74651413e+01],
       [  4.35734153e-01],
       [  9.33829229e-03],
       [ -1.06622010e-01],
       [  6.44106984e-01],
       [ -4.25131839e-06],
       [ -3.77322501e-03],
       [ -4.26648885e-01],
       [ -4.40514028e-01]], dtype=float32)

Compare with pure NumPy

In [21]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


Compare with Scikit-Learn

In [22]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


## 5. Using Batch Gradient Descent
정규방정식 대신 Batch Gradient Descent를 사용<br>

gradient를 수동으로 계산하고 텐서플로의 autodiff 기능을 사용해 텐서플로가 gradient를 계산하도록 한다. 


In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [24]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[  1.00000000e+00   6.60969987e-17   5.50808322e-18   6.60969987e-17
  -1.06030602e-16  -1.10161664e-17   3.44255201e-18  -1.07958431e-15
  -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ..., -0.06612179 -0.06360587
  0.01359031]
0.111111111111
(20640, 9)


### Manually computing the gradients

random_uniform() : numpy rand()함수와 마찬가지로 주어진 shape와 value 범위안에서 무작위 값을 포함하는 텐서를 만들 그래프의 노드를 생성<br>
assign() : 새로운 값을 변수에 할당 할 노드를 생성한다. 이 경우에 Batch Gradient Descent step을 수행한다. ($\theta^{(nest step)}=\theta-\eta\nabla_\theta MSE(\theta)$)<br>

이때 main loop는 training step을 계속해서 수행한다.(n_epochs times)<br>
100회때마다 현재의 Mean Square Error(MSE)를 출력한다. 

In [25]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.714501
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.555572
Epoch 400 MSE = 0.548812
Epoch 500 MSE = 0.543636
Epoch 600 MSE = 0.539629
Epoch 700 MSE = 0.536509
Epoch 800 MSE = 0.534068
Epoch 900 MSE = 0.532147


반복수행 횟수가 증가할수록 MSE값은 감소한다.

In [26]:
best_theta

array([[ 2.06855249],
       [ 0.88740271],
       [ 0.14401658],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393812],
       [-0.04269557],
       [-0.66145277],
       [-0.63752776]], dtype=float32)

### Using autodiff
위의 코드는 정상적으로 작동하지만 비용함수(MSE)에서 gradient를 수학적으로 유도해야한다. <br>
신경망 네트워크를 사용할 때 이 작업은 매우 어렵고 오류가 발생하기 쉽다. <br>

텐서플로에는 autodiff 기능으로 gradient를 자동으로 효율적으로 계산할 수 있다.  <br>
`gradients = ...` 이 부분을 `gradients = tf.gradients(mse,[theta])[0]` 이렇게 바꿔주면 된다. 

gradient() 함수는 op(mse)와 변수 목록을 취하고, 각 op에 대한 gradient를 계산하기 위해 하나의 변수 당 하나의 ops 목록을 만든다.  <br>
텐서플로는 reverse-mode autodiff를 사용한다. 많은 input과 적은 output이 있을 때 효과적이다. 

In [27]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [28]:
gradients = tf.gradients(mse, [theta])[0]

In [29]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.714501
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.555572
Epoch 400 MSE = 0.548812
Epoch 500 MSE = 0.543636
Epoch 600 MSE = 0.539629
Epoch 700 MSE = 0.536509
Epoch 800 MSE = 0.534068
Epoch 900 MSE = 0.532147
Best theta:
[[ 2.06855249]
 [ 0.88740271]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145277]
 [-0.6375277 ]]


How could you find the partial derivatives of the following function with regards to `a` and `b`?

In [30]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [31]:
my_func(0.2, 0.3)

-0.21253923284754914

In [32]:
reset_graph()

a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")
for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)

grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

Let's compute the function at $a=0.2$ and $b=0.3$, and the partial derivatives at that point with regards to $a$ and with regards to $b$:

In [33]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.212537
[-1.1388494, 0.19671395]


### Using a `GradientDescentOptimizer`
텐서플로는 다양한 optimizer 도구를 제공한다.  <br>
`gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)`<br>

이 부분을 바꿔준다. <br>

`optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)`<br>

만약 다른 유형의 optimizer를 사용하려면 한줄만 바꾸면 된다.<br>
`optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)`<br>


In [30]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [31]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [32]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.714501
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.555572
Epoch 400 MSE = 0.548812
Epoch 500 MSE = 0.543636
Epoch 600 MSE = 0.539629
Epoch 700 MSE = 0.536509
Epoch 800 MSE = 0.534068
Epoch 900 MSE = 0.532147
Best theta:
[[ 2.06855249]
 [ 0.88740271]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145277]
 [-0.6375277 ]]


### Using a momentum optimizer

In [37]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [38]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9) # 이 한줄만 변경해주면 된다.

In [39]:
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [40]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Best theta:
[[ 2.06855798]
 [ 0.82962859]
 [ 0.11875337]
 [-0.26554456]
 [ 0.30571091]
 [-0.00450251]
 [-0.03932662]
 [-0.89986444]
 [-0.87052065]]


# 6. Feeding data to the training algorithm
mini-batch gradient를 수행해보자. <br>
`placeholder()`를 사용하여 매 반복마다 x,y를 다음 mini-batch로 교체해준다.<br>

이 노드는 실제로 계산을 수행하지 않는다. 런타임시 출력하도록 지정한 데이터만 출력한다. <br>
주로 텐서플로에 학습 데이터를 전달하는데 사용된다. `placeholder`에 값을 지정하지 않으면 exception이 발생한다.<br>

`placeholder()`함수에는 텐서의 타입과 shape를 지정할 수 있다. <br>
차원에 `None`을 지정하면 `any size`를 의미한다. <br>
Example. 노드 A : `A = tf.placeholder(tf.float32, shape=(None, 3))`<br>
노드 B : `B=A+5` <br>
B를 평가할 때, A값을 명시하는 eval() 메소드에 `feed_dict`로 A값을 전달한다. 

## Placeholder nodes

In [33]:
reset_graph()

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)

[[ 6.  7.  8.]]


In [42]:
print(B_val_2)

[[  9.  10.  11.]
 [ 12.  13.  14.]]


## Mini-batch Gradient Descent
Mini-batch를 구현하기 위해서 기존의 코드중 x,y를 변경한다. <br>
`X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")` <br>

그다음 batch size와 batches의 전체 개수를 계산한다.  <br>
`batch_size=100
n_batches=int(np.ceil(m/batch_size))` <br>

In [34]:
n_epochs = 1000
learning_rate = 0.01

In [35]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [36]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [37]:
n_epochs = 10

In [38]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [39]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})  
            # execution phase에서 하나씩 mini-batches로 가져와 feed_dict파라미터를 통해 X 및 y값을 제공

    best_theta = theta.eval()

In [40]:
best_theta

array([[ 2.07033372],
       [ 0.86371452],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.80304712]], dtype=float32)

# 7. Saving and restoring a model
모델을 학습 한 후 파라미터를 디스크에 저장하여 언제든지 다시 불러와 다른 프로그램에서 사용하고, 다른 모델과 비교하는 등의 작업을 수행해야 한다.  <br>
또 학습도중 일정 간격으로 체크포인트를 저장하여 컴퓨터가 다운됐을 때 체크포인트부터 계속할 수도 있다.  <br>
텐서플로는 모델을 쉽게 저장, 복원 할 수 있다. <br>

모든 변수 노드를 생성한 후 construction phase 마지막 부분에 saver 노드를 만든다.  <br>
그다음 execution phase에서 체크포인트 파일의 세션과 경로를 전달하여 모델을 저장할 때마다 단지 save()메소드를 호출하면 된다.

In [50]:
reset_graph()

n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer() # init 노드를 사용해 변수 초기화
saver = tf.train.Saver()  # saver 노드 생성

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "/tmp/my_model.ckpt")   # 파일 저장
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.714501
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.555572
Epoch 400 MSE = 0.548812
Epoch 500 MSE = 0.543636
Epoch 600 MSE = 0.539629
Epoch 700 MSE = 0.536509
Epoch 800 MSE = 0.534068
Epoch 900 MSE = 0.532147


In [51]:
best_theta

array([[ 2.06855249],
       [ 0.88740271],
       [ 0.14401658],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.66145277],
       [-0.6375277 ]], dtype=float32)

모델을 restore하는 것도 with 블록에 `sess.run(init)`대신 생성한 saver노드에 `restore()` 메소드를 호출하면 된다. 

In [52]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [53]:
np.allclose(best_theta, best_theta_restored)

True

saver는 모든 변수를 자체 이름으로 저장 및 복원하지만 더 많은 제어가 필요할 경우 <br>
저장/복원할 변수와 사용할 이름을 지정할 수 있다. <br>
saver는 'weights' 이름 아래 'theta' 변수만을 저장하거나 복원한다.

In [54]:
saver = tf.train.Saver({"weights": theta})

기본적으로 save() 메소드는 동일한 이름의 .meta 확장자를 가진 두번째 파일에 그래프 구조를 저장한다.<br>
`tf.train.import_meta_graph()`를 사용해 이 그래프 구조를 로드할 수 있다. <br>
이렇게하면 기본 그래프에 그래프가 추가되고 saver 인스턴스가 반환되어 그래프의 상태(즉, 변수값)을 복원하는데 사용 가능하다.

In [55]:
reset_graph()
# notice that we start with an empty graph.

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # meta 파일을 로드하여 그래프 구조를 저장
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # 기본 파일을 저장하여 그래프 상태를 불러옴
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [56]:
np.allclose(best_theta, best_theta_restored)

True

그래프 구조와 변수 값을 모두 포함하여 저장된 모델을 검색하는 과정 없이 저장된 모델을 완전히 복원할 수 있다.

# 9. Visualizing the graph and Training Curves using TensoBoard
선형회귀를 Mini-batch를 이용하여 학습하는 계산 그래프를 얻었고 주기적으로 체크포인트를 저장하였다. <br>
이젠 TensorBoard를 사용하여 결과를 시각적으로 표현하고자 한다. <br>
학습통계를 feed하여 웹브라우저에서 시각적으로 표시할 수 있다. <br>
그래프의 정의(graph's definition)를 제공하고 이를 통해 탐색할 수 있는 인터페이스도 제공한다. <br>
그래프의 오류를 식별하고 병목현상을 찾는 등의 작업에 매우 유리하다. 


## inside Jupyter

In [41]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [42]:
show_graph(tf.get_default_graph())

## Using TensorBoard
텐서보드가 읽을 로그 디렉토리에 그래프 정의 및 학습통계(MSE)를 기록한다. <br>
프로그램을 실행할 때마다 다른 로그 디렉터리를 사용하지 않으면 텐서보드가 다른 실행의 통계를 병합하여 시각화를 엉망으로 만든다.<br>
가장 간단하게 로그 디렉토리 이름에 timestamp를 포함시킨다.

In [43]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [44]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

MSE를 평가하고, summary라는 텐서보드 호환 바이너리 로그 문자열에 쓰는 노드를 그래프에 만든다.<br>
두번째는 로그 디렉토리의 로그 파일에 summary를 쓰는데 사용할 FileWriter를 만든다. <br>
첫번째 매개변수는 로그 디렉토리의 경로, 두번째는 시각화하려는 그래프이다.

In [45]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [46]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

생성시 FileWriter는 로그디렉토리가 아직 존재하지 않는 경우는 로그파일을 작성하고 eventfile이라는 이진 로그파일에 그래프 정의를 작성한다.<br>
학습중 정기적으로 mse_summary 노드를 평가하기위해 실행 단계를 업데이트 해야한다. <br>


In [47]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:   # 10개의 mini-batch 마다 시행 (mse_summary 평가)
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     # not shown

In [48]:
file_writer.close()

In [49]:
best_theta

array([[ 2.07033372],
       [ 0.86371452],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.80304712]], dtype=float32)

# 10. Name scopes
신경망 처럼 복잡한 모델을 다룰때 그래프는 수천개의 노드로 복잡해 보일 수 있다. <br>
이를 피하기 위해 이름 범위를 만들어 노드를 그룹화 할 수 있다. <br>


In [66]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

"loss"라는 이름 범위 내에서 error와 mse를 정의할 수 있다<br>


In [67]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [68]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [69]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.07033372]
 [ 0.86371452]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.80304712]]


정의한 각 op의 앞에는 loss/ 가 붙는다.

In [70]:
print(error.op.name)

loss/sub


In [71]:
print(mse.op.name)

loss/mse


In [72]:
reset_graph()

a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="a")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name)

a
a_1
param/a
param_1/a


# 11. Modularity
두 개의 ReLU(Rectified Linear Unit)의 출력을 더하는 그래프를 만들고 싶다고 가정해보자<br>


An ugly flat code:

In [73]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

이러한 반복적인 코드는 유지가 어렵고 오류발생이 쉽다. <br>
텐서플로는 DRY(Don't Repeat Yourself) 상태를 유지할 수 있다. <br>

5개의 ReLU 함수를 만들고 합계를 출력해보자

In [74]:
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [75]:
file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

노드를 만들때 텐서플로는 해당 이름이 있는지 확인한 후 "_" 추가하면서 노드의 고유성을 부여한다. <br>
첫번째 ReLU는 weights, bias, z, relu 가 포함되고<br>
두번째 ReLU는 weights_1, bias_1, z_1, relu_1이라는 노드가 포함된다. 
<img src='9-3.PNG'>

아래는 name_scope를 사용한 코드이다.

In [76]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    # not shown
        b = tf.Variable(0.0, name="bias")                             # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                      # not shown
        return tf.maximum(z, 0., name="max")                          # not shown

In [77]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()

Name_scope를 이용하게 되면 나타나는 그래프는 다음과 같다.
<img src='9-4.PNG'>

## 12. Sharing Variables
그래프의 다양한 컴포넌트 사이에서 변수를 공유하기 위해 간단한 함수를 작성하고 필요로하는 함수에 파라미터를 전달한다. <br>
모든 ReLU의 treshold 변수를 공유하기위해 사용되는 ReLU threshold를 제어한다고 생각할 때 변수를 만들고 relu() 함수에 전달만 하면 된다.

Sharing a `threshold` variable the classic way, by defining it outside of the `relu()` function then passing it as a parameter:

In [78]:
reset_graph()

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

많은 공유 파라미터가 있는 경우는 파라미터로 전달해야 하는 것은 힘들다.<br>
많은 사람들이 자신의 모델에 있는 모든 변수를 포함하는 파이선 딕셔너리를 만들고 모든 함수에 전달한다. <br>
또 다른 사람들은 각 모듈에 대한 클래스를 작성한다. <br>

아래처럼 첫번째 호출시 공유 파라미터를 relu()함수의 속성으로 설정할수도 있다. 

In [79]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, relu.threshold, name="max")

In [80]:
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

텐서플로는 또 다른 옵션을 제공한다. <br>
아직 존재하지 않는 파라미터를 `get_variable()` 함수를 사용하여 공유 파라미터를 작성하거나 이미 존재하는 경우 이를 재사용한다. <br>
원하는 행위는 현재 `variable_scope()`의 특성에 의해 제어된다. <br>

예) 아래 코드에서 relu/ threshold 라는 이름의 변수를 만든다

In [81]:
reset_graph()

with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))

기존의 relu/threshold 변수를 가져오거나 (fetch), 존재하지 않을 경우(get_variable()을 이용하지 않았을 경우)는 예외를 발생시킨다. <br>
대신 scope의 reuse_variable()메소드를 호출하여 블록 내에서 reuse속성을 true로 설정할 수 있다. 

In [82]:
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

In [83]:
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

이제 relu() 함수가 파라미터로 전달하지 않고도 threshold 변수에 액세스 할 수 있는 모든 요소가 있다.<br>
먼저 relu() 함수를 정의하고, relu/threshold 변수를 만들고 relu()함수를 호출하여 5개의 ReLU를 작성한다. <br>
relu()함수는 relu/threshold 변수를 재사용하고 다른 ReLU 노드를 만든다.

In [84]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

In [85]:
file_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
file_writer.close()

<img src='9-5.PNG'>

In [86]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu"):
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("", default_name="") as scope:   
    first_relu = relu(X)     # create the shared variable
    scope.reuse_variables()  # then reuse it
    relus = [first_relu] + [relu(X) for i in range(4)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu8", tf.get_default_graph())
file_writer.close()

첫번째 호출 시 relu()함수 내에 threshold 변수를 만든 다음, 다음 호출에서 이를 다시 사용한다. <br>
relu() 함수는 name scope 또는 변수 공유에 대해 걱정할 필요가 없게 된다. <br>
get_variable()만 호출하면 threshold 변수를 만들거나 재사용한다. 

In [87]:
reset_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [88]:
file_writer = tf.summary.FileWriter("logs/relu9", tf.get_default_graph())
file_writer.close()

<img src='9-6.PNG'>